# Problem 3

Use this notebook to write your code for problem 3.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3D - Convolutional network

As in problem 2, we have conveniently provided for your use code that loads, preprocesses, and deals with the uglies of the MNIST data.

In [2]:
# load MNIST data into Keras format
import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
# look at the shapes
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
# we'll need to one-hot encode the labels
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)

In [5]:
# don't forget to NORMALIZE
x_train = np.divide(x_train, 255)
x_test = np.divide(x_test, 255)

In [6]:
# we must reshape the X data (add a channel dimension)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
x_test = x_test.reshape(tuple(list(x_test.shape) + [1]))

In [7]:
# look at the shapes
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers

# sample model
# note: what is the difference between 'same' and 'valid' padding?
# Take a look at the outputs to understand the difference, or read the Keras documentation!
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.10))

model.add(Conv2D(8, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.10))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [17]:
# why don't we take a look at the layers and outputs
# note: `None` in the first dimension means it can take any batch_size!
for i in range(len(model.layers)):
    layer = model.layers[i]
    print(layer)
    print(layer.output_shape)

(None, 28, 28, 8)
(None, 28, 28, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 14, 14, 8)
(None, 7, 7, 8)
(None, 7, 7, 8)
(None, 392)
(None, 64)
(None, 64)
(None, 10)
(None, 10)
(None, 10)


In [18]:
# our model has some # of parameters:
model.count_params()

26506

In [19]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
# Train the model, iterating on the data in batches of 32 samples
history = model.fit(x_train, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 39s 648us/step - loss: 0.4127 - acc: 0.9181 - val_loss: 0.1255 - val_acc: 0.9761
Epoch 2/10
60000/60000 [==============================] - 36s 606us/step - loss: 0.1627 - acc: 0.9608 - val_loss: 0.0578 - val_acc: 0.9835
Epoch 3/10
60000/60000 [==============================] - 37s 610us/step - loss: 0.1182 - acc: 0.9701 - val_loss: 0.0542 - val_acc: 0.9857
Epoch 4/10
60000/60000 [==============================] - 37s 612us/step - loss: 0.0947 - acc: 0.9747 - val_loss: 0.0434 - val_acc: 0.9867
Epoch 5/10
60000/60000 [==============================] - 37s 613us/step - loss: 0.0853 - acc: 0.9768 - val_loss: 0.0404 - val_acc: 0.9873
Epoch 6/10
60000/60000 [==============================] - 37s 615us/step - loss: 0.0783 - acc: 0.9782 - val_loss: 0.0424 - val_acc: 0.9876
Epoch 7/10
60000/60000 [==============================] - 37s 615us/step - loss: 0.0721 - acc: 0.9797 - val_loss: 0.

Above, we output the training loss/accuracy as well as the validation (here, the TEST) loss and accuracy. To confirm that these are right, we can explicitly print out the training and test losses/accuracies.

In [21]:
# note that our model outputs two eval params:
# 1. loss (categorical cross-entropy)
# 2. accuracy
model.metrics_names

['loss', 'acc']

In [22]:
model.evaluate(x=x_train, y=y_train)

60000/60000 [==============================] - 15s 246us/step


[0.025796938360032315, 0.99231666666666662]

In [23]:
model.evaluate(x=x_test, y=y_test)

10000/10000 [==============================] - 2s 249us/step


[0.034721569664590057, 0.9889]

In [26]:
new_drops = np.linspace(0, 1, 10)
for drop in new_drops:
    model = Sequential()
    model.add(Conv2D(8, (3, 3), padding='same',
                     input_shape=(28, 28, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop))

    model.add(Conv2D(8, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    # For a multi-class classification problem
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    print("Drop probability: " + str(drop))
    history = model.fit(x_train, y_train, epochs=1, batch_size=32,
                        validation_data=(x_test, y_test))

Drop probability: 0.0
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 34s 561us/step - loss: 0.3530 - acc: 0.9398 - val_loss: 0.0964 - val_acc: 0.9806
Drop probability: 0.111111111111
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 37s 617us/step - loss: 0.4269 - acc: 0.9136 - val_loss: 0.1138 - val_acc: 0.9789
Drop probability: 0.222222222222
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 38s 635us/step - loss: 0.4960 - acc: 0.8892 - val_loss: 0.1408 - val_acc: 0.9736
Drop probability: 0.333333333333
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 39s 645us/step - loss: 0.5795 - acc: 0.8581 - val_loss: 0.1494 - val_acc: 0.9737
Drop probability: 0.444444444444
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [=====================

Not bad! Let's see if you can do better.